In [1]:
import glob
import os
import re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Aw yeah, yeah', "I mean there's a lot of women out here that's just, you know", 'They just defensive in so many things, man']


In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> aw yeah , yeah <end>',
 '<start> i mean there s a lot of women out here that s just , you know <end>',
 '<start> they just defensive in so many things , man <end>',
 '<start> you missing so many key nutrients in your life <end>',
 '<start> i , i just want you to stay healthy in these streets <end>',
 '<start> oh , no no <end>',
 '<start> somebody gotta give you some vitamin d , you know what i mean ? <end>',
 '<start> yeah , luda she got them handlebars <end>',
 '<start> gotta keep a tight grip on that donkey <end>',
 '<start> she be givin out battle scars <end>']

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen =15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 1571   49 ...    0    0    0]
 [  65   16    9 ...    7   34    3]
 [   2   45   35 ...    0    0    0]
 ...
 [   2   11   16 ...    0    0    0]
 [   2    8  175 ...    0    0    0]
 [   2    5   91 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7efc71daf110>


print(tensor[:3, :10])

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

In [5]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 1571   49    4   49    3    0    0    0    0    0    0    0    0]
[1571   49    4   49    3    0    0    0    0    0    0    0    0    0]


In [6]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val =train_test_split(src_input,tgt_input
                                                 ,test_size=0.2, random_state=42)

In [7]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
enc_train

Source Train: (140599, 14)
Target Train: (140599, 14)


array([[  28,  131,   10, ...,    6, 6942, 1559],
       [   2,    7,   32, ...,    0,    0,    0],
       [   2,   24,    6, ...,    0,    0,    0],
       ...,
       [   2,  134,   71, ...,  203,    3,    0],
       [   2,   30,   46, ...,    0,    0,    0],
       [   2,   41,   21, ...,    0,    0,    0]], dtype=int32)


BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

#dataset2 = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
#dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
#dataset2

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


In [9]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)


model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train,dec_train, epochs=10, validation_data = (enc_val, dec_val))

Epoch 1/10
4394/4394 [==============================] - 138s 31ms/step - loss: 3.2783 - val_loss: 2.9859
Epoch 2/10
4394/4394 [==============================] - 139s 32ms/step - loss: 2.8021 - val_loss: 2.7745
Epoch 3/10
4394/4394 [==============================] - 139s 32ms/step - loss: 2.5047 - val_loss: 2.6512
Epoch 4/10
4394/4394 [==============================] - 139s 32ms/step - loss: 2.2433 - val_loss: 2.5853
Epoch 5/10
4394/4394 [==============================] - 139s 32ms/step - loss: 2.0160 - val_loss: 2.5491
Epoch 6/10
4394/4394 [==============================] - 139s 32ms/step - loss: 1.8188 - val_loss: 2.5368
Epoch 7/10
4394/4394 [==============================] - 139s 32ms/step - loss: 1.6489 - val_loss: 2.5359
Epoch 8/10
4394/4394 [==============================] - 140s 32ms/step - loss: 1.5029 - val_loss: 2.5506
Epoch 9/10
4394/4394 [==============================] - 139s 32ms/step - loss: 1.3810 - val_loss: 2.5818
Epoch 10/10
4394/4394 [==============================] 

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [12]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

validation_loss 가 2.6 정도 나와서 embedding 을 512 로 조정하였더니 오히려 더 안좋은 결과가 나왔습니다. 제가 시도해본것들은:

1. Embedding size 512 로 바꿈
2. fit 안에 enc_training, dec_training 대신 dataset 으로 묶어서 학습 (validation data 또한 묶음)
3. enc_training dec_training 으로 학습
4. hidden_size 를 2048 로 바꿈 --> 결과가 더 좋았지만...노트북 고장남..

결국 노트북이 고장났습니다..블랙아웃되었습니다..ㅠ